In [ ]:
import qiskit

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal import jigsaw

from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.benchmarks.bv import bv_circuit
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal import non_standard_coupling_maps

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

In [ ]:
n_shots = 8000
n_rounds = 5

circuit_res = []
aim_res = []
sim_res = []
full_res = []
lin_res = []
tpf_res = []
tpf_e_res = []
jr_res = []

backend_generators = [
    partial(Grid, 2, 2), # 4
    partial(Grid, 3, 2), # 6
    partial(Grid, 4, 2), # 8
    partial(Grid, 3, 3), # 9
    partial(Grid, 5, 2), # 10
    partial(Grid, 6, 2), # 12
    partial(Grid, 7, 2), # 14
    partial(Grid, 5, 3), # 15
    partial(Grid, 4, 4)  # 16
]
circuit_generators = [state_prep_circuits.plus_state_prep]

pb = Progressbar(20, 5 * n_rounds * len(backend_generators), 'Sims')

for backend_gen in backend_generators:
    
    circuit_round = []
    aim_round = []
    sim_round = []
    tpf_round = []
    tpf_e_round = []
    jr_round = []

    for r_num in range(n_rounds):
            
        backend = backend_gen()
        n_qubits = len(backend.properties().qubits)
        err_cmap = non_standard_coupling_maps.error_coupling_map(backend=backend)
        
        # Build tensor patches
        tpf = TensorPatchFitter(backend, n_shots=n_shots / 2)
        tpf.build()
        
        tpf_e = TensorPatchFitter(backend, n_shots=n_shots / 2, coupling_map=err_cmap)
        tpf_e.build()
        
        full = qiskit_full(backend, n_qubits, n_shots)
        linear = qiskit_linear(backend, n_qubits, n_shots)

        # Run Circuits
        circuit = state_prep_circuits.plus_state_prep(backend)      
        pb.tick('Bare: {}'.format(n_qubits))
        
        # Transpile and run circuit
        tc = qiskit.transpile(circuit,
                              backend,
                              optimization_level=0,
                              initial_layout=list(range(n_qubits))
                             )
        cr = qiskit.execute(circuit, 
                             backend, 
                             shots=n_shots / 2, 
                             optimization_level=0,
                             initial_layout=list(range(n_qubits))
                            ).result().get_counts()
        circuit_round.append(state_prep_circuits.plus_state_dist(cr))

        # Apply AIM to circuit
        pb.tick('AIM {}'.format(n_qubits))
        ar = aim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)
        aim_round.append(state_prep_circuits.plus_state_dist(ar))

        # Apply SIM to circuit
        pb.tick('SIM {}'.format(n_qubits))
        sr = sim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True) 
        sr = renormalise_measurement_results(sr, 1)
        sim_round.append(state_prep_circuits.plus_state_dist(sr))

        ### DO NOT RUN FOR LARGE NUMBERS OF QUBITS (>8-10)
        ### WILL TAKE FOREVER, THEN OOM
        # Apply Full
#         pb.tick('Full {}'.format(n_qubits))
#         fr = full.apply(cr)
#         full_round.append(state_prep_circuits.plus_state_dist(fr))

#         # Apply Linear
#         pb.tick('Lin {}'.format(n_qubits))
#         lr = linear.apply(cr)
#         linear_round.append(state_prep_circuits.plus_state_dist(lr))

        # Apply CMC to circuit
        pb.tick('CMC {}'.format(n_qubits))
        tr = tpf.apply(cr)
        tpf_round.append(state_prep_circuits.plus_state_dist(tr))
        
        # Apply CMC to circuit
        pb.tick('CMC E{}'.format(n_qubits))
        tr = tpf_e.apply(cr)
        tpf_e_round.append(state_prep_circuits.plus_state_dist(tr))

        # Jigsaw
        pb.tick('JIG {}'.format(n_qubits))
        jr = jigsaw.jigsaw(circuit, backend, n_shots, equal_shot_distribution=True)
        jr_round.append(state_prep_circuits.plus_state_dist(jr))
        print(jr_round)
        
    circuit_res.append(circuit_round)
    aim_res.append(aim_round)
    sim_res.append(sim_round)
        #full_res.append(full_round)
        #lin_res.append(linear_round)
    tpf_res.append(tpf_round)
    tpf_e_res.append(tpf_e_round)
    jr_res.append(jr_round)

results = {
'bare':circuit_res,
'aim':aim_res,
'sim':sim_res,
'full':full_res,
'lin':lin_res,
'cmc':tpf_res,
'cmc_err':tpf_e_res,
'jig':jr_res
}

In [ ]:
results

In [ ]:
grid_4 = {'bare': [[0.22125,
   0.15300000000000002,
   0.182,
   0.20024999999999998,
   0.22200000000000003]],
 'aim': [[0.22016666666666662,
   0.16483333333333333,
   0.21150000000000002,
   0.2048333333333333,
   0.22916666666666669]],
 'sim': [[0.22637500396138044,
   0.17674999509815853,
   0.19837500005406034,
   0.20812499972264048,
   0.2257500026519409]],
 'full': [[0.029772395538069374,
   0.00548873737686012,
   0.009025200698214997,
   0.013019727085288813,
   0.01806880269221134]],
 'lin': [[0.02164976970227206,
   0.009157647374358369,
   0.0047316049073457545,
   0.0246055134527558,
   0.02383792877867469]],
 'cmc': [[0.0415745895480642,
   0.03556951310841683,
   0.08475805676491704,
   0.06929545667038284,
   0.04091712414443227]],
 'cmc_err': [[0.07215324855774796,
   0.05621097476266601,
   0.041295416513364436,
   0.038327956499125626,
   0.05040115485901764]],
 'jig': [[0.1463069091933184,
   0.11434178072618412,
   0.1325293384202837,
   0.12478349423097085,
   0.15016764200576221]]}

In [ ]:
grid_6 = {'bare': [[0.27175,
   0.27375000000000005,
   0.31275,
   0.28475,
   0.31224999999999997]],
 'aim': [[0.26183333333333336,
   0.27166666666666667,
   0.30333333333333334,
   0.2823333333333334,
   0.31416666666666665]],
 'sim': [[0.2642499934094016,
   0.280874999806183,
   0.2908750051561473,
   0.29025000362164777,
   0.3192500080288516]],
 'full': [[0.05702174138579874,
   0.04150243077991361,
   0.06893290147075098,
   0.035548297134961104,
   0.04357789753581698]],
 'lin': [[0.05589549320091586,
   0.03857060669354534,
   0.06754564282250286,
   0.04168968744542462,
   0.03964376293644756]],
 'cmc': [[0.09235610951707407,
   0.14657576396325317,
   0.09676886567906218,
   0.07956083822747223,
   0.08137608153400067]],
 'cmc_err':[[0.07471119960295558,
   0.062222380229434715,
   0.09770178255219253,
   0.09655853343043036,
   0.0968834126929643]],
 'jig': [[0.16665909731485362,
   0.2194058670910794,
   0.2033072326321087,
   0.16694387979831427,
   0.2408963762683763]]}

In [ ]:
grid_8 = {
    'bare': [[0.39325, 0.384, 0.3585, 0.40075, 0.39775]],
 'aim': [[0.3995, 0.37733333333333335, 0.365, 0.4055, 0.39949999999999997]],
 'sim': [[0.39099999914962974,
   0.3793749932551241,
   0.37962499806868083,
   0.3996249988629589,
   0.3997499982181144]],
 'full': [[0.07492761054176378,
   0.08232543446166041,
   0.0376145204554546,
   0.058486103762973585,
   0.07866355920837509]],
 'lin': [[0.0705519444588289,
   0.08036422117125419,
   0.0485408473537029,
   0.06326659044990163,
   0.07196727696074628]],
 'cmc': [[0.15660892603997778,
   0.14115815072539106,
   0.11383491002329238,
   0.1449464955327902,
   0.16516712757807706]],
 'cmc_err': [[0.1291328543311246,
   0.12187114719311631,
   0.11365282046551911,
   0.12200200984225812,
   0.13614499602058389]],
 'jig': [[0.3160183670094996,
   0.23250761964867084,
   0.30039176829470066,
   0.26167304349961473,
   0.26468659481447854]]}


In [ ]:
grid_9 = {'bare': [[0.413, 0.4375, 0.4205, 0.43424999999999997, 0.43975]],
 'aim': [[0.40549999999999997,
   0.4443333333333333,
   0.4248333333333333,
   0.44333333333333336,
   0.4166666666666667]],
 'sim': [[0.4094999989405333,
   0.4374999999354259,
   0.43837499613167913,
   0.4460000042951722,
   0.42662500291485583]],
 'full': [[0.08083561545742035,
   0.07206177524325008,
   0.07351385161401647,
   0.06554680653384715,
   0.09619123556768894]],
 'lin': [[0.07148787302629295,
   0.0751823772365085,
   0.06364788575127839,
   0.05908104058507391,
   0.09276307599869033]],
 'cmc': [[0.1443503211371423,
   0.16067978860646032,
   0.1536452658091615,
   0.15186926546192836,
   0.1626044466802049]],
 'cmc_err': [[0.15628912306827714,
   0.13871836891775297,
   0.11964896240457568,
   0.15060643235394094,
   0.14887353457321262]],
 'jig': [[0.3177383628147963,
   0.3361893654018128,
   0.31329107848001464,
   0.3225970918843147,
   0.3181264331170125]]}
grid_10 = {'bare': [[0.4625,
   0.46749999999999997,
   0.45525,
   0.42200000000000004,
   0.45475]],
 'aim': [[0.45816666666666667,
   0.465,
   0.4676666666666667,
   0.41133333333333333,
   0.45916666666666667]],
 'sim': [[0.4552499946481988,
   0.47662499272775893,
   0.46624999939384,
   0.41424999338794843,
   0.4457499975515967]],
 'full': [[0.08504235810422361,
   0.06556519420810636,
   0.06886219422351458,
   0.09682301331278931,
   0.08659222521018484]],
 'lin': [[0.09166428429967144,
   0.06960869663058927,
   0.05755247211723835,
   0.08857163008515934,
   0.08917958932266035]],
 'cmc': [[0.18847838962696128,
   0.18409256903453575,
   0.17188243593767077,
   0.19578905156722898,
   0.18961250345182717]],
 'cmc_err':[[0.1725720465524146,
   0.15848531913501013,
   0.1800545298037941,
   0.21252913769324583,
   0.149497678337345]],
 'jig': [[0.3790265223099666,
   0.2990879299862757,
   0.3337117458034285,
   0.25394536269880147,
   0.33744299426275404]]}
grid_12 = {'bare': [[0.46, 0.503, 0.48724999999999996, 0.57775, 0.522]],
 'aim': [[0.44749999999999995,
   0.517,
   0.47783333333333333,
   0.5673333333333334,
   0.5258333333333334]],
 'sim': [[0.46262500934973627,
   0.5118750070438725,
   0.490499998505402,
   0.5781250029144757,
   0.5245000012710006]],
 'full': [[]],
 'lin': [[]],
 'cmc': [[0.23141298574518865,
   0.21965786283628413,
   0.22739206511749932,
   0.26350491358518724,
   0.24919087517018562]],
 'cmc_err': [[0.2179550012480267,
   0.25004947837049873,
   0.27800974124343414,
   0.22326541421818613,
   0.20805332967128998]],
 'jig': [[0.3325732679028284,
   0.39228210874936875,
   0.36054701202468736,
   0.42525098246075055,
   0.36233302693139613]]}

grid_14 = {'bare': [[0.56525, 0.58375, 0.5745, 0.5437500000000001, 0.601]],
 'aim': [[0.5740000000000001,
   0.5898333333333333,
   0.5595,
   0.5388333333333333,
   0.6096666666666666]],
 'sim': [[0.5711249943539518,
   0.5836249943848855,
   0.5798749975588109,
   0.537625003583771,
   0.6058749936684584]],
 'full': [],
 'lin': [],
 'cmc': [[0.28753437446908087,
   0.30036978789265495,
   0.3010259983800489,
   0.2737649352148623,
   0.3089434370770596]],
 'cmc_err':[[0.33835768789996695,
   0.29240863742680395,
   0.2789005399687562,
   0.2551899249813221,
   0.2843322986960108]],
 'jig': [[0.42451121836632977,
   0.4342311481156372,
   0.4321613947096443,
   0.4338375581330553,
   0.5256739213197792]]}
grid_15={'bare': [[0.5895, 0.5855, 0.55725, 0.576, 0.58175]],
 'aim': [[0.5863333333333334,
   0.5866666666666667,
   0.5808333333333333,
   0.6006666666666667,
   0.5803333333333334]],
 'sim': [[0.5876250014361375,
   0.5988749931189654,
   0.5839999971343206,
   0.6041249982263126,
   0.5741249987637113]],
 'full': [[]],
 'lin': [[]],
 'cmc': [[0.33655222272845076,
   0.3251393392200211,
   0.31791463779369855,
   0.30803378251491675,
   0.32617949404583396]],
 'cmc_err': [[0.2557977143744942,
   0.3135410581141455,
   0.30611787697891996,
   0.3323702813308279,
   0.3437474042747702]],
 'jig': [[0.4384581458052078,
   0.4183680526150075,
   0.44293363829150095,
   0.5002423341753162,
   0.4300023420623164]]}
grid_16={'bare': [[0.641, 0.62375, 0.63925, 0.65625, 0.5922499999999999]],
 'aim': [[0.644,
   0.6371666666666667,
   0.632,
   0.6536666666666667,
   0.6113333333333333]],
 'sim': [[0.654500005553862,
   0.6309999987344782,
   0.625625000721111,
   0.6536249979576079,
   0.6180000010063813]],
 'full': [[]],
 'lin': [[]],
 'cmc': [[0.3749411796899539,
   0.3479555510214219,
   0.39478394911499054,
   0.38671014233264867,
   0.3280510593954451]],
 'cmc_err': [[0.3796521365639531,
   0.3597317846241992,
   0.3589030819370159,
   0.3474565981175553,
   0.34802265705033586]],
 'jig': [[0.5524413985174622,
   0.5281299324455435,
   0.49561731535864273,
   0.4978030866641916,
   0.42257077439800356]]}